<a href="https://colab.research.google.com/github/aviramberg276/personality/blob/develop/personality_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# import hypertools as hyp
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf

sb.set(style="darkgrid")
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
database = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/personality/Aviram_dataset_wide_A.csv", index_col=[0])

Seperate classes and data

In [0]:
classes = database[database.columns[1:5]]
data = database[database.columns[5:]]

In [0]:
data = data.fillna(data.mean())

In [5]:
classes

,AG,Compassion,Respectfulness,Trust
1,5.083333,5.00,5.00,5.25
2,5.000000,6.50,6.00,2.50
3,5.333333,5.50,5.00,5.50
4,4.750000,5.75,5.25,3.25
5,4.416667,3.00,5.25,5.00
...,...,...,...,...
221,4.166667,5.50,3.75,3.25
222,4.833333,6.75,5.00,2.75
223,6.666667,6.75,6.75,6.50
224,5.083333,5.75,5.25,4.25


In [6]:
classes.values.min(), classes.values.max()

(1.0, 7.0)

create a loss function that depends on the rules of each trait for example: if a=5 b = b * a/2 ... 

In [7]:
import keras.backend as K
def custom_loss(layer):

    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        return K.mean(K.mean(y_true, axis=0), axis=-1)
    return loss

val = 0
def loss3(y_true,y_pred):
    global val
    val= K.sum(K.sqrt(y_true -y_pred))
    # print(K.sum((y_true -y_pred)**2))
    return K.sum((y_true -y_pred)**2)

def loss2(y_true,y_pred, num):
#         return K.mean(K.square(y_pred - y_true) + K.square(layer), axis=-1)
    return (np.abs((y_true - y_test)) < num).sum() / (y_test.shape[0] * y_test.shape[1]) * 100

def mapping_to_target_range( x, target_min=1, target_max=7) :
    x02 = K.tanh(x) + 1 # x in range(0,2)
    scale = ( target_max-target_min )/2.
    return  x02 * scale + target_min


Using TensorFlow backend.


In [0]:
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import optimizers, regularizers

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(1500, input_dim=2910, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(Dropout(0.15, seed=np.random.seed(seed)))
    model.add(Dense(500, input_dim=1000, kernel_initializer='normal', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(Dense(200, input_dim=500, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(Dropout(0.15, seed=np.random.seed(seed)))
    model.add(Dense(100, input_dim=160, kernel_initializer='normal', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(Dense(40, input_dim=100, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(Dropout(0.15, seed=np.random.seed(seed)))
    model.add(Dense(4, kernel_initializer='normal', activation=mapping_to_target_range))
    # Compile model
    # optimizer = optimizers.Adam(lr=0.0001)
    optimizer = optimizers.Adam(lr=0.0001)
    model.compile(loss=loss3, optimizer=optimizer, metrics=['accuracy'])
    return model

In [0]:
# def baseline_model():
# 	# create model
# 	model = Sequential()
# 	model.add(Dense(4, input_dim=2910, kernel_initializer='normal', activation='relu'))
# 	model.add(Dense(4, kernel_initializer='normal'))
# 	# Compile model
# 	model.compile(loss=loss3, optimizer='adam')
# 	return model

In [0]:
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.compat.v1.keras.backend import clear_session
from tensorflow.compat.v1.keras.backend import get_session
# import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del estimator
        del clf# this is from global space - change this as you need
    except:
        pass

#     print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
#     config = tensorflow.ConfigProto()
#     config.gpu_options.per_process_gpu_memory_fraction = 1
#     config.gpu_options.visible_device_list = "0"
#     set_session(tensorflow.Session(config=config))
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus: 
        tf.config.experimental.set_memory_growth(gpu, True)

In [0]:
reset_keras()

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scalar = StandardScaler()
data2 = scalar.fit_transform(data)
data2 = data

In [0]:

from keras.callbacks import ModelCheckpoint
seed = 7
# checkpoint
# filepath="/content/drive/My Drive/Colab Notebooks/data/personality/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, mode='max')
# callbacks_list = [checkpoint]

In [0]:
log_dir="/content/drive/My Drive/Colab Notebooks/model/personality/reg/"
checkpoint_acc = ModelCheckpoint(log_dir + 'drop50_ep{epoch:03d}-loss{loss:.3f}-accuracy{accuracy:.3f}.h5',
        monitor='accuracy', save_weights_only=True, save_best_only=True, period=3, mode='max')
checkpoint_loss = ModelCheckpoint(log_dir + 'drop50_ep{epoch:03d}-accuracy{accuracy:.3f}-val_accuracy{val_accuracy:.3f}.h5',
        monitor='val_accuracy', save_weights_only=True, save_best_only=True, period=3, mode='max')
# callbacks_list = [checkpoint_acc]
callbacks_list = [checkpoint_acc, checkpoint_loss]

In [0]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=500, batch_size=10, verbose=0, callbacks=callbacks_list,shuffle=True)
# estimator.fit(x_train, y_train, nb_epoch=10000, batch_size=10, verbose=0, callbacks=callbacks_list, shuffle=True, validation_split=0.2)
estimator.fit(scalar.fit_transform(data), classes, nb_epoch=100000, batch_size=5, verbose=0, callbacks=callbacks_list, shuffle=True, validation_split=0.3)

/usr/local/lib/python3.6/dist-packages/keras/wrappers/scikit_learn.py:151: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  history = self.model.fit(x, y, **fit_args)


In [0]:
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import optimizers

def baseline_model():
    # create model
    model = Sequential()
    # model.add(Dense(1500, input_dim=2910, kernel_initializer='normal', activation='relu'))
    # model.add(Dropout(0.3, seed=np.random.seed(seed)))
    # model.add(Dense(500, input_dim=1000, kernel_initializer='normal'))
    # model.add(Dense(200, input_dim=1000, kernel_initializer='normal', activation='relu'))
    model.add(Dense(170, input_dim=225, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3, seed=np.random.seed(seed)))
    # model.add(Dense(100, input_dim=160, kernel_initializer='normal'))
    model.add(Dense(40, input_dim=100, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.4, seed=np.random.seed(seed)))
    model.add(Dense(4, kernel_initializer='normal', activation=mapping_to_target_range))
    # Compile model
    # optimizer = optimizers.Adam(lr=0.0001)
    optimizer = optimizers.Adam(lr=0.0001)
    model.compile(loss=loss3, optimizer=optimizer, metrics=['accuracy'])
    return model
log_dir="/content/drive/My Drive/Colab Notebooks/model/personality/pca/"
checkpoint_acc = ModelCheckpoint(log_dir + 'drop50_ep{epoch:03d}-loss{loss:.3f}-accuracy{accuracy:.3f}.h5',
        monitor='accuracy', save_weights_only=True, save_best_only=True, period=3, mode='max')
checkpoint_loss = ModelCheckpoint(log_dir + 'drop50_ep{epoch:03d}-accuracy{accuracy:.3f}-val_accuracy{val_accuracy:.3f}.h5',
        monitor='val_accuracy', save_weights_only=True, save_best_only=True, period=3, mode='max')
# callbacks_list = [checkpoint_acc]
callbacks_list = [checkpoint_acc, checkpoint_loss]

In [0]:
log_dir="/content/drive/My Drive/Colab Notebooks/model/personality/"
checkpoint = ModelCheckpoint(log_dir + 'try2_ep{epoch:03d}-loss{loss:.3f}-accuracy{accuracy:.3f}.h5',
        monitor='accuracy', save_weights_only=True, save_best_only=True, period=3, mode='max')
callbacks_list = [checkpoint]

In [0]:
x_val = scalar.fit_transform(data)[-50:]
y_val = classes[-50:]
x_train = scalar.fit_transform(data)[:-50]
y_train = classes[:-50]
pca = PCA(n_components=225)
estimator = KerasRegressor(build_fn=baseline_model, epochs=500, batch_size=10, verbose=0, callbacks=callbacks_list,shuffle=True)
# estimator.fit(x_train, y_train, nb_epoch=10000, batch_size=10, verbose=0, callbacks=callbacks_list, shuffle=True, validation_split=0.2)
estimator.fit(pca.fit_transform(scalar.fit_transform(data)), classes, nb_epoch=100000, batch_size=5, verbose=0, callbacks=callbacks_list, shuffle=True, validation_split=0.3)

/usr/local/lib/python3.6/dist-packages/keras/wrappers/scikit_learn.py:151: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  history = self.model.fit(x, y, **fit_args)


In [0]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=500, batch_size=10, verbose=1, callbacks=callbacks_list,shuffle=True)
estimator.fit(scalar.fit_transform(data), classes, nb_epoch=10000, batch_size=10, verbose=1, callbacks=callbacks_list, shuffle=True)

/usr/local/lib/python3.6/dist-packages/keras/wrappers/scikit_learn.py:151: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  history = self.model.fit(x, y, **fit_args)


Streaming output truncated to the last 5000 lines.
225/225 [==============================] - 0s 462us/step - loss: 4.1445 - accuracy: 0.8622
Epoch 7502/10000
225/225 [==============================] - 0s 468us/step - loss: 3.9087 - accuracy: 0.8578
Epoch 7503/10000
225/225 [==============================] - 0s 519us/step - loss: 3.9102 - accuracy: 0.8533
Epoch 7504/10000
225/225 [==============================] - 0s 479us/step - loss: 3.9560 - accuracy: 0.8578
Epoch 7505/10000
225/225 [==============================] - 0s 486us/step - loss: 4.1547 - accuracy: 0.8489
Epoch 7506/10000
225/225 [==============================] - 0s 492us/step - loss: 3.7888 - accuracy: 0.8889
Epoch 7507/10000
225/225 [==============================] - 0s 513us/step - loss: 4.0058 - accuracy: 0.8222
Epoch 7508/10000
225/225 [==============================] - 0s 512us/step - loss: 3.9557 - accuracy: 0.8533
Epoch 7509/10000
225/225 [==============================] - 0s 467us/step - loss: 3.9138 - accuracy: 0

In [0]:
from sklearn.model_selection import GroupKFold
import random

group_kfold = GroupKFold(n_splits=15)
group_kfold.get_n_splits(data2, classes)
gkf = list(group_kfold.split(data2, classes, database[database.columns[0]]))
estimator = KerasRegressor(build_fn=baseline_model, epochs=500, batch_size=10, verbose=0, callbacks=callbacks_list)
outcomes = []
# fold = 0
seed = 7
best = []

for i in range(1000):
  mid_outcomes = []
  print("Iteration: " + str(i))
  random.shuffle(gkf)
  fold =0
  for train_index, test_index in gkf:
    fold += 1 
    X_train, X_test = scalar.fit_transform(data2.values[train_index]), scalar.fit_transform(data2.values[test_index])
    y_train, y_test = classes.values[train_index], classes.values[test_index]
#         clf.fit(X_train, y_train)
    estimator.fit(X_train, y_train)
    predictions = estimator.predict(X_test)
#         accuracy = accuracy_score(y_test, predictions)
    accuracy = np.mean(np.sqrt((predictions - y_test) ** 2))
    
#     precent = sum(np.sqrt((predictions - y_test) ** 2) > 1) / sum(np.sqrt((predictions - y_test) ** 2)) * 100
    precent = loss2(predictions, y_test, 0.5)
    accuracy = loss2(predictions, y_test, 1)
    if accuracy > 80:
      best.append((train_index, test_index))
    # outcomes.append(accuracy)
    print("Fold {0} accuracy: {1} above 1 error precent: {2}".format(fold, accuracy, precent))
    mid_outcomes.append(accuracy)
  outcomes.append(np.mean(mid_outcomes))
  print(predictions)
#     break
mean_outcome = np.mean(outcomes)
print("Mean Accuracy: {0}".format(mean_outcome)) 


Iteration: 0


KeyError: ignored

In [0]:
import random
random.shuffle(gkf)

In [0]:
X_train

array([[-0.23151286, -0.15404112, -0.11945799, ..., -0.80491643,
        -0.46151267, -0.89143811],
       [ 0.04977536, -0.15415687, -0.14387222, ..., -0.80296615,
        -0.52736392,  1.65512753],
       [-0.09574923, -0.13952304, -0.13166142, ..., -1.28662487,
         0.04214682,  0.0168052 ],
       ...,
       [-0.13267029, -0.10616259, -0.10155903, ..., -1.64480515,
        -0.06975462,  0.6323478 ],
       [-0.08913987, -0.13320756, -0.11785971, ..., -0.95762538,
        -0.37312782,  2.5265282 ],
       [-0.21517457, -0.15559052, -0.12182597, ..., -1.034588  ,
        -0.88065114,  1.64456259]])

In [0]:
gkf

[(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
          26,  27,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
          40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
          53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
          66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
          79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95,  96,  97,  98,  99, 100, 102, 103, 104, 105,
         106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
         119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
         132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
         145, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
         159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
         172, 173, 174, 175, 176, 177,

In [0]:
estimator.model

In [0]:
|data2.values[train_index].shape

(202, 2910)

In [0]:
data2.shape

(225, 2910)

In [0]:
model = baseline_model()
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]

In [0]:
K.learning_phase?

In [0]:
K.learning_phase()

<tf.Tensor 'learning_phase:0' shape=() dtype=int32>

In [0]:
K.function??

In [0]:
X_train[0].shape

(2910,)

In [0]:
layer_outs = [func([X_train[0], K.learning_phase()]) for func in functors]
print(layer_outs)

_SymbolicException: ignored

In [0]:
model = estimator.model
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp], [out]) for out in outputs]

In [0]:
layer_outs = [func([X_train]) for func in functors]
print(layer_outs)

[[array([[0.        , 0.        , 5.2434616 , ..., 0.        , 0.20762572,
        0.        ],
       [2.2706046 , 3.7941692 , 0.3229487 , ..., 0.        , 1.4527415 ,
        0.        ],
       [1.0201445 , 0.        , 0.        , ..., 4.6925626 , 0.        ,
        2.3362944 ],
       ...,
       [2.0308568 , 0.24314162, 5.6879554 , ..., 0.        , 2.93006   ,
        0.        ],
       [2.5652559 , 0.15287124, 2.5245836 , ..., 4.941967  , 2.6387713 ,
        1.9576228 ],
       [3.6854773 , 2.2450893 , 1.0256157 , ..., 0.        , 3.5285814 ,
        0.6602014 ]], dtype=float32)], [array([[0.        , 0.        , 5.2434616 , ..., 0.        , 0.20762572,
        0.        ],
       [2.2706046 , 3.7941692 , 0.3229487 , ..., 0.        , 1.4527415 ,
        0.        ],
       [1.0201445 , 0.        , 0.        , ..., 4.6925626 , 0.        ,
        2.3362944 ],
       ...,
       [2.0308568 , 0.24314162, 5.6879554 , ..., 0.        , 2.93006   ,
        0.        ],
       [2.56525

In [0]:
layer_outs[-1][0].shape

(221, 4)

In [0]:
|np.argwhere(np.isnan(X_train))

array([[128, 320],
       [128, 321],
       [128, 322],
       [128, 323],
       [128, 324],
       [128, 325],
       [128, 326],
       [128, 327],
       [128, 328],
       [128, 329],
       [128, 330],
       [128, 331],
       [128, 332],
       [128, 333],
       [128, 334],
       [151, 320],
       [151, 321],
       [151, 322],
       [151, 323],
       [151, 324],
       [151, 325],
       [151, 326],
       [151, 327],
       [151, 328],
       [151, 329],
       [151, 330],
       [151, 331],
       [151, 332],
       [151, 333],
       [151, 334],
       [152, 320],
       [152, 321],
       [152, 322],
       [152, 323],
       [152, 324],
       [152, 325],
       [152, 326],
       [152, 327],
       [152, 328],
       [152, 329],
       [152, 330],
       [152, 331],
       [152, 332],
       [152, 333],
       [152, 334],
       [183, 320],
       [183, 321],
       [183, 322],
       [183, 323],
       [183, 324],
       [183, 325],
       [183, 326],
       [183,

In [0]:
zgbcjkg

function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}

setInterval(ClickConnect, 60000)